In [1]:
# Cellular Automaton for a simulation of vaccine efficacy

''' The basis of this algorithm comes from this paper: https://www.nature.com/articles/nature04153#MOESM1

-----------Concept-----------

    There exists a grid, 90x60 (3:2 aspect ratio is pretty).
    
    Each cell in the grid has three states:
        uninfected
        infected, contagious
        infected, non-contagious
        
        -infected cells are more saturated the higher viral load they contain.
        
    A cell can spread to each of its neighbors described by a radius n (simulates lockdowns; fewer neighbor contact).
    
    Cases can worsen at the end of each generation, after spreading is calculated.
    
    The Virus class needs two peices of data to construct an effective simulation:
        -value 'R0' represents the effective reproductive number; essentially how many other people are infected on average.
        -value 'k' represents the dispersion parameter; how variable R0 is.
    
    The person class is a shell that needs to distinguish between vaccinated and unvaccinated (base) child classes.
        -attribute 'Rt', which calculates it's current reproductive probability.
        -A person has an attribute 'viral_load'. Higher viral load increases Rt (R0 at given time t).
            -After N generations, a person is considered to be non-contaigous.
        -attribute 'tolerance', when viral_load > tolerance at end of generation, cell dies.
          
    **Vaccinated Person: probability of breakthrough case, resistance constant for viral load calculation.
        
-----------Math-----------
    
    The Poisson distribution is the probability of x independent event occurrences in λ, the mean occurrence per interval
    -Not a good fit for disease spread, models a homogenous population.
    -One use: Poisson(R0)=Z (the expected secondary case number at nth percentile = Z^n); Identify SSE (super spreader events).
    
    Inverse Square Law, while usually reserved for physics, is a way to simulate the decrease of viral load over distance.
    -This is because this simulation has no enviroment to create enviromental variables.
        Amount is proportional to 1/D^2.
    
    Negative Binomial Formula: With x trials, results in r successes, and probability of success P:

        b*(x; r, P) = x-1Cr-1 * Pr * (1 - P)x - r   OR    { (x-1)! / [ (r-1)!(x-r)!] } * Pr * (1 - P)x - r
    
    The geometric distribution is a type of negative binomial distribution...
        -with 'x' as the number of trials required for a single success, and P the probability of success: g(x; P) = P * Qx - 1
        

-----------Implementation-----------

1st Goal: create a working spread model, no vaccinated persons, use binary numbers

*NON EDGE CASES:
    
    Each infected cell generates a random percentage S (spread), influenced by its viral load. 
    Each infected cell also generates an Rt, using k and R0.
    Poisson(Rt) is calculated for all x, 0-N(max neighbors).
    Infected neighbors are chosen by distance using Z = int(Poisson(Rt)==S) # rounding
    One generation is simulated; tolerance influences viral load, using a geometric distribution
    Cells that have been infected for K weeks can no longer infect
    Cells with low enough viral load lose the sickness.
    Non-Contagious cells are marked.
    
    RULESET:
        If the cell is infected:
            Z[count] = ele for count, ele in enumerate(poisson.pmf([0,1,2,3,4,5,6,7,8], Rt)) # list comprehension to make
            
'''

'''git clone REP_ADDRESS
    cd into the repo
    git status (should be red)
    if you made changes, git add file
        git status (should be green)
        git commit -m 'describe what was done'
        git push
    git pull, syncs up with the remote server
    git push '''

In [3]:
# importing modules
from scipy.stats import poisson
from scipy.stats import nbinom
import numpy as np
import matplotlib.pyplot as plt

# for debug
np.set_printoptions(suppress=True)
import time

#global vars?

# classes
class Cell():
    def __init__(self, viral_load = 0, tolerance = 5): # tolerance TBI
        self.viral_load = viral_load
        self.tolerance = tolerance
        
    def __int__(self):
        return self.viral_load

class InfectedCell(Cell):
    def __init__(self, viral_load = None, tolerance = None, age = None):
        super().__init__(size, viral_load, tolerance)
        self.age = age

class Board():
    
    def __init__(self, row, col):
        self.row = row
        self.col = col
    
        '''these two need to be instance variables, board is a numpy array of cell objects,
        and disp_board is the int/float version of the cell instance variable "viral load"
        but they arent working outside the init function '''
        board = np.ndarray(shape=(self.row, self.col),dtype=np.object)
        disp_board = np.zeros(shape=(self.row, self.col))
        for i in range(row):
            for j in range(col):
                board[i, j] = Cell()
    
    def display(self):
        '''this block breaks, either because the board variable isnt correctly an instance attribute, or im not calling it
            as such properly, or because the numpy array board is not subscriptable'''
        #for i in range(self.row):
        #    for j in range(self.col):
        #        disp_board[i, j] = self.board    ##when i typed board[i, j], it was usually not subscriptable
        print(self.disp_board)

# This is where my spread alg starts, I eventually want it to take a radius argument.
def prob_mat_construct():
    prob_mat = np.zeros([9, 4])# 4x9 mat, each column is the probability that j neighbors are infected in i generations.
    n = [x for x in range(10)] # represents the amount of neighbors infected in one go. n sucesses
    n.remove(0)
    p = 0.14 # probability of spread upon contact
    for i in range(9):
        x = np.arange(nbinom.ppf(0.01, n[i], p), nbinom.ppf(0.99, n[i], p)) # nbinom distribution
        z = nbinom.pmf(x, n[i], p)
        for count, ele in enumerate(x): # throwing away data over 4 attempts (simulates non-contagious periods)
            if ele >= 4: # number of generations until non contagious, corresponds with the shape of prob_mat
                continue
            elif z[count] < 0.0005: # the great filter
                continue
            else:
                prob_mat[i, int(ele)] = round((z[count]*100), 2)
    return prob_mat
    
# main
def main():
    # constants
    r_naught = 3
    

    prob_mat = prob_mat_construct()
    cell_board = Board(90,60)
    
    cell_board.display()
    '''this section creates a probability guide for the virus'''
    
    print(prob_mat)
    '''this stuff was for displaying the board and timing that process'''
    #t0 = time.time()
    #plt.figure(figsize = (60,90))
    #plt.imshow(cell_board.display(), interpolation='nearest')
    #t1 = time.time()
    #total = t1-t0
    #print(total)
    
if __name__ == "__main__":
    main()

AttributeError: 'Board' object has no attribute 'disp_board'